In [1]:
from ontology_v6 import graph, getOnto

from owlready2 import *
import random

from util import save_as_csv, replace_relations, identify_logical_operators

onto = getOnto()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


ok-onto6
ok-onto6


In [2]:

def generate_all_question_results(templates):
    with onto:
        all_classes = list(onto.classes())
        all_foods = list(onto.Food.instances())

        question_results = []
        for ont_class in all_classes:
            if onto.Food in ont_class.ancestors():
                for random_food in ont_class.instances():
                    for related_taste in random_food.has_taste:
                        for related_color in random_food.has_color:
                            template = random.choice(templates)

                            question_result = template.replace('{class}', ont_class.name)
                            question_result = question_result.replace('{food}', random_food.name)
                            question_result = question_result.replace('{taste}', related_taste.name)
                            question_result = question_result.replace('{colour}', related_color.name)
                            
                            # has_taste = [food.name for food in all_foods if related_taste in food.has_taste]
                            # is_class = [food.name for food in all_foods if isinstance(food, ont_class)]
                            # is_colour = [food.name for food in all_foods if related_color in food.has_color]
                            # fa = [food for food in has_taste if food in is_class and food in is_colour and food is not random_food.name]
                            fa = f"{random_food.name} is a {ont_class.name} that has a {related_taste.name} taste and is {related_color.name}, so it satisfies the requirements."

                            question_results.append((question_result, fa))

    return question_results

templates = ["Suppose I want a {taste} tasting {class} that is also {colour} coloured, why is {food} a good choice?"]
            # "If {food} is not suitable for me, what other {class} with {colour} colour and {taste} taste can I choose?",
            # "Looking for alternatives to {food}, recommend a {colour} {class} that has a {taste} flavor."]
results = generate_all_question_results(templates)

save_as_csv(results, "question_results_why_1.csv", ["Question", "fa"])


Results saved to question_results_why_1.csv


In [2]:
# 11. Multi Fruit operators 
from util import replace_relations, identify_logical_operators

def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        all_relations = all_tastes + all_colours + all_shapes + all_nutritions
        ops = ["and", "or", "and not", "or not"]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            for rel_3, attributes_3 in all_relations:
                                if rel_3 != rel_1 and rel_3 != rel_2:
                                    for attribute3 in attributes_3:

                                        template = random.choice(templates)
                                        op1 = random.choice(ops)
                                        op2 = random.choice(ops)
                                        
                                        question_result = template.replace('{rel_a}', rel_1)
                                        question_result = question_result.replace('{attr_a}', attribute1.name)
                                        question_result = question_result.replace('{rel_b}', rel_2)
                                        question_result = question_result.replace('{attr_b}', attribute2.name)
                                        question_result = question_result.replace('{rel_c}', rel_3)
                                        question_result = question_result.replace('{attr_c}', attribute3.name)
                                        question_result = question_result.replace('{op_a}', op1)
                                        question_result = question_result.replace('{op_b}', op2)

                                        def getMatches(rel, attr, op):
                                            if "not" in op:
                                                return [food.name for food in all_fruits if attr not in getattr(food, rel, [])]
                                            else:
                                                return [food.name for food in all_fruits if attr in getattr(food, rel, [])]

                                        def reduceAns(set1, set2, op):
                                            if "and" in op:
                                                return [item for item in set1 if item in set2]
                                            else:
                                                return list(set(set1) | set(set2))

                                        match1 = getMatches(rel_1, attribute1, "")
                                        match2 = getMatches(rel_2, attribute2, op1)
                                        match3 = getMatches(rel_3, attribute3, op2)

                                        step1 = reduceAns(match1, match2, op1)
                                        fa = reduceAns(step1, match3, op2)

                                        if not fa: 
                                            continue

                                        f = random.choice(fa)
                                        rel_desc_1 = f"{rel_1} {attribute1.name}"
                                        rel_desc_2 = f"{'not ' if 'not' in op1 else ''}{rel_2} {attribute2.name}"
                                        rel_desc_3 = f"{'not ' if 'not' in op2 else ''}{rel_3} {attribute3.name}"

                                        m1 = rel_desc_1 if f in match1 else ""
                                        m2 = rel_desc_2 if f in match2 else ""
                                        m3 = rel_desc_3 if f in match3 else ""
                                        
                                        ans_logic = f"{' and '.join(part for part in [m1, m2, m3] if part)}"
                                        ans_logic_nl = replace_relations(ans_logic)

                                        if "and" in op1 and "and" in op2:
                                            ans = f"The question uses conjunctive logic for which {f} satisfies all three required conditions as it {ans_logic_nl}"
                                        elif "or" in op1 and "or" in op2:
                                            ans = "The question uses disjunctive logic"
                                            if f in match1 and f in match2 and f in match3:
                                                ans += f" for which {f} satisfies all three required conditions as it {ans_logic_nl}"
                                            elif f in match1 and f in match2:
                                                ans += f" and {f} meets the first two conditions as it {ans_logic_nl}"
                                            elif f in match1 and f in match3:
                                                ans += f" and {f} meets the first and last conditions as it {ans_logic_nl}"
                                            elif f in match2 and f in match3:
                                                ans += f" and {f} meets the last two conditions as it {ans_logic_nl}"
                                            elif f in match1:
                                                ans += f" and while {f} does not meet the second or third conditions, it {ans_logic_nl}, so satisfies the first condition of the question"
                                            elif f in match2:
                                                ans += f" and while {f} does not meet the first or third conditions, it {ans_logic_nl}, so satisfies the second condition of the question"
                                            elif f in match3:
                                                ans += f" and while {f} does not meet either of the first two conditions, it {ans_logic_nl}, so satisfies the last condition of the question"
                                            else:
                                                ans += f" and while {f} does not meet two of the three conditions, it {ans_logic_nl}, so satisfies the conditions of the question"
                                        else:
                                            ans = f"The question contains both conjunctive and disjunctive logic"
                                            if "or" in op1:
                                                if not match1:
                                                    ans += f", and since no fruit satisfies the first part of the 'or' condition, we consider the second part for which {f} meets the requirements as it {ans_logic_nl}"
                                                elif not match2:
                                                    ans += f", and while no fruit satisfies the combination of {rel_desc_2} and {rel_desc_3}, {f} meets the first criterion of the 'or' condition as it {ans_logic_nl}"
                                                elif f in match1:
                                                    ans += f" and {f} {ans_logic_nl} meaning it meets the first condition of the 'or' statement, as well as the 'and' requirement"
                                                else:
                                                     ans += f", where {f} satisfies the 'and' condition as it {ans_logic_nl}"
                                            elif "or" in op2:
                                                if not match3:
                                                    ans += f", and no fruit satisfies the latter part of the 'or' condition, though {f} meets the first requirements as it both {ans_logic_nl}"
                                                elif not match1 or not match2:
                                                    ans += f", and while no fruit satisfies the combination of {rel_desc_1} and {rel_desc_2}, {f} meets the latter criterion of the 'or' condition as it {rel_desc_3}"
                                                elif f in match3:
                                                    ans += f" and {f} {ans_logic_nl} meaning it meets the last condition"
                                                else:
                                                     ans += f", where {f} satisfies the 'and' condition as it {ans_logic_nl}"

                                        ans_nl = replace_relations(ans)

                                        question_result = question_result.replace('{food}', f)
                                        qnl = replace_relations(question_result)
                                        operators, level, types = identify_logical_operators(question_result)

                                        question_results.append((question_result, qnl, ans_nl, match1, match2, match3, fa, f, operators, level, types))
                    
    return question_results
# Since no fruit satisfies the combination of being bitter {rl_1} and green {rl_2} based on the first part of the AND {op_1} condition,
# we consider the second part of the OR {op_2}condition, which is the shape Banana satisfies the 'Long shape' {rl_3}criterion.

templates = [
    # "Which fruit has taste {taste} and has shape {shape} or has colour {colour}",
    "Why is {food} a good choice when I want a fruit that {rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} {rel_c} {attr_c}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_why_2c.csv", [
    "Question",
    "questions_nlp",
    "answer_reason",
    "a1",
    "a2",
    "a3",
    "fa",
    "keyword_ans",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_why_2c.csv


In [3]:
# 11. Veg version
from util import replace_relations, identify_logical_operators

def generate_all_question_results(templates):
    with onto:
        all_veg = list(onto.Vegetable.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        all_relations = all_tastes + all_colours + all_shapes + all_nutritions
        ops = ["and", "or", "and not", "or not"]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            for rel_3, attributes_3 in all_relations:
                                if rel_3 != rel_1 and rel_3 != rel_2:
                                    for attribute3 in attributes_3:

                                        template = random.choice(templates)
                                        op1 = random.choice(ops)
                                        op2 = random.choice(ops)
                                        
                                        question_result = template.replace('{rel_a}', rel_1)
                                        question_result = question_result.replace('{attr_a}', attribute1.name)
                                        question_result = question_result.replace('{rel_b}', rel_2)
                                        question_result = question_result.replace('{attr_b}', attribute2.name)
                                        question_result = question_result.replace('{rel_c}', rel_3)
                                        question_result = question_result.replace('{attr_c}', attribute3.name)
                                        question_result = question_result.replace('{op_a}', op1)
                                        question_result = question_result.replace('{op_b}', op2)

                                        def getMatches(rel, attr, op):
                                            if "not" in op:
                                                return [food.name for food in all_veg if attr not in getattr(food, rel, [])]
                                            else:
                                                return [food.name for food in all_veg if attr in getattr(food, rel, [])]

                                        def reduceAns(set1, set2, op):
                                            if "and" in op:
                                                return [item for item in set1 if item in set2]
                                            else:
                                                return list(set(set1) | set(set2))

                                        match1 = getMatches(rel_1, attribute1, "")
                                        match2 = getMatches(rel_2, attribute2, op1)
                                        match3 = getMatches(rel_3, attribute3, op2)

                                        step1 = reduceAns(match1, match2, op1)
                                        fa = reduceAns(step1, match3, op2)

                                        if not fa: 
                                            continue

                                        f = random.choice(fa)
                                        rel_desc_1 = f"{rel_1} {attribute1.name}"
                                        rel_desc_2 = f"{'not ' if 'not' in op1 else ''}{rel_2} {attribute2.name}"
                                        rel_desc_3 = f"{'not ' if 'not' in op2 else ''}{rel_3} {attribute3.name}"

                                        m1 = rel_desc_1 if f in match1 else ""
                                        m2 = rel_desc_2 if f in match2 else ""
                                        m3 = rel_desc_3 if f in match3 else ""
                                        
                                        ans_logic = f"{' and '.join(part for part in [m1, m2, m3] if part)}"
                                        ans_logic_nl = replace_relations(ans_logic)

                                        if "and" in op1 and "and" in op2:
                                            ans = f"The question uses conjunctive logic for which {f} satisfies all three required conditions as it {ans_logic_nl}"
                                        elif "or" in op1 and "or" in op2:
                                            ans = "The question uses disjunctive logic"
                                            if f in match1 and f in match2 and f in match3:
                                                ans += f" for which {f} satisfies all three required conditions as it {ans_logic_nl}"
                                            elif f in match1 and f in match2:
                                                ans += f" and {f} meets the first two conditions as it {ans_logic_nl}"
                                            elif f in match1 and f in match3:
                                                ans += f" and {f} meets the first and last conditions as it {ans_logic_nl}"
                                            elif f in match2 and f in match3:
                                                ans += f" and {f} meets the last two conditions as it {ans_logic_nl}"
                                            elif f in match1:
                                                ans += f" and while {f} does not meet the second or third conditions, it {ans_logic_nl}, so satisfies the first condition of the question"
                                            elif f in match2:
                                                ans += f" and while {f} does not meet the first or third conditions, it {ans_logic_nl}, so satisfies the second condition of the question"
                                            elif f in match3:
                                                ans += f" and while {f} does not meet either of the first two conditions, it {ans_logic_nl}, so satisfies the last condition of the question"
                                            else:
                                                ans += f" and while {f} does not meet two of the three conditions, it {ans_logic_nl}, so satisfies the conditions of the question"
                                        else:
                                            ans = f"The question contains both conjunctive and disjunctive logic"
                                            if "or" in op1:
                                                if not match1:
                                                    ans += f", and since no vegetable satisfies the first part of the 'or' condition, we consider the second part for which {f} meets the requirements as it {ans_logic_nl}"
                                                elif not match2:
                                                    ans += f", and while no vegetable satisfies the combination of {rel_desc_2} and {rel_desc_3}, {f} meets the first criterion of the 'or' condition as it {ans_logic_nl}"
                                                elif f in match1:
                                                    ans += f" and {f} {ans_logic_nl} meaning it meets the first condition of the 'or' statement, as well as the 'and' requirement"
                                                else:
                                                     ans += f", where {f} satisfies the 'and' condition as it {ans_logic_nl}"
                                            elif "or" in op2:
                                                if not match3:
                                                    ans += f", and no vegetable satisfies the latter part of the 'or' condition, though {f} meets the first requirements as it both {ans_logic_nl}"
                                                elif not match1 or not match2:
                                                    ans += f", and while no vegetable satisfies the combination of {rel_desc_1} and {rel_desc_2}, {f} meets the latter criterion of the 'or' condition as it {rel_desc_3}"
                                                elif f in match3:
                                                    ans += f" and {f} {ans_logic_nl} meaning it meets the last condition"
                                                else:
                                                     ans += f", where {f} satisfies the 'and' condition as it {ans_logic_nl}"

                                        ans_nl = replace_relations(ans)

                                        question_result = question_result.replace('{food}', f)
                                        qnl = replace_relations(question_result)
                                        operators, level, types = identify_logical_operators(question_result)

                                        question_results.append((question_result, qnl, ans_nl, match1, match2, match3, fa, f, operators, level, types))
                    
    return question_results
# Since no fruit satisfies the combination of being bitter {rl_1} and green {rl_2} based on the first part of the AND {op_1} condition,
# we consider the second part of the OR {op_2}condition, which is the shape Banana satisfies the 'Long shape' {rl_3}criterion.

templates = [
    # "Which veg has taste {taste} and has shape {shape} or has colour {colour}",
    "Why is {food} a good choice when I want a vegetable that {rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} {rel_c} {attr_c}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_why_veg.csv", [
    "Question",
    "questions_nlp",
    "answer_reason",
    "a1",
    "a2",
    "a3",
    "fa",
    "keyword_ans",
    "logical_operator",
    "logical_level",
    "logical_types"
])


Results saved to question_results_why_veg.csv


In [ ]:
# 11. Multi Fruit operators 
from util import replace_relations, identify_logical_operators

def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_nutritions = [("has_nutrition", list(onto.Nutrition.instances()))]
        all_tastes = [("has_taste", list(onto.Taste.instances()))]
        all_colours = [("has_color", list(onto.Color.instances()))]
        all_shapes = [("has_shape", list(onto.Shape.instances()))]

        all_relations = all_tastes + all_colours + all_shapes + all_nutritions
        ops = ["and", "or", "and not", "or not"]

        question_results = []

        for rel_1, attributes_1 in all_relations:
            for attribute1 in attributes_1:
                for rel_2, attributes_2 in all_relations:
                    if rel_2 != rel_1:
                        for attribute2 in attributes_2:
                            for rel_3, attributes_3 in all_relations:
                                if rel_3 != rel_1 and rel_3 != rel_2:
                                    for attribute3 in attributes_3:

                                        template = random.choice(templates)
                                        op1 = random.choice(ops)
                                        op2 = random.choice(ops)
                                        
                                        question_result = template.replace('{rel_a}', rel_1)
                                        question_result = question_result.replace('{attr_a}', attribute1.name)
                                        question_result = question_result.replace('{rel_b}', rel_2)
                                        question_result = question_result.replace('{attr_b}', attribute2.name)
                                        question_result = question_result.replace('{rel_c}', rel_3)
                                        question_result = question_result.replace('{attr_c}', attribute3.name)
                                        question_result = question_result.replace('{op_a}', op1)
                                        question_result = question_result.replace('{op_b}', op2)

                                        def getMatches(rel, attr, op):
                                            if "not" in op:
                                                return [food.name for food in all_fruits if attr not in getattr(food, rel, [])]
                                            else:
                                                return [food.name for food in all_fruits if attr in getattr(food, rel, [])]

                                        def reduceAns(set1, set2, op):
                                            if "and" in op:
                                                return [item for item in set1 if item in set2]
                                            else:
                                                return list(set(set1) | set(set2))

                                        match1 = getMatches(rel_1, attribute1, "")
                                        match2 = getMatches(rel_2, attribute2, op1)
                                        match3 = getMatches(rel_3, attribute3, op2)

                                        step1 = reduceAns(match1, match2, op1)
                                        fa = reduceAns(step1, match3, op2)

                                        if not fa: 
                                            continue

                                        f = random.choice(fa)
                                        m1 = f"{rel_1} {attribute1.name}" if f in match1 else ""
                                        m2 = f"{rel_2} {attribute2.name}" if f in match2 else ""
                                        m2 = f"not {m2}" if "not" in op1 and m2 else m2
                                        m3 = f"{rel_3} {attribute3.name}" if f in match3 else ""
                                        m3 = f"not {m3}" if "not" in op2 and m3 else m3
                                        # ans = f"{f} {m1} {m2} {m3}"
                                        ans = f"{f} {' and '.join(part for part in [m1, m2, m3] if part)}"
                                        ans_nl = replace_relations(ans)

                                        question_result = question_result.replace('{food}', f)
                                        qnl = replace_relations(question_result)
                                        operators, level, types = identify_logical_operators(question_result)

                                        question_results.append((question_result, qnl, ans_nl, match1, match2, match3, fa, operators, level, types))
                    
    return question_results

templates = [
    # "Which fruit has taste {taste} and has shape {shape} or has colour {colour}",
    "Why is {food} a good choice when I want a fruit that {rel_a} {attr_a} {op_a} {rel_b} {attr_b} {op_b} {rel_c} {attr_c}?"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_why_meat.csv", [
    "Question",
    "questions_nlp",
    "answer",
    "a1",
    "a2",
    "a3",
    "fa",
    "logical_operator",
    "logical_level",
    "logical_types"
])


In [6]:
# Why - random fruit + veg
def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_vegetables = list(onto.Vegetable.instances())

        question_results = []
        unique_fa_sets = set()

        for food in ["beef", "lamb", "pork", "fish", "shrimp", "salmon"]:
            for _ in range(50):
                template = random.choice(templates)

                question_result = template.replace('{food}', food)

                num_fruit = random.randint(0, 5)
                num_veg = 5 - num_fruit
                fruits = random.sample(all_fruits, num_fruit)
                vegs = random.sample(all_vegetables, num_veg)

                fa = [food.name for food in fruits + vegs]
                fa_set = frozenset(fa)  # Using frozenset to make it hashable

                if fa_set not in unique_fa_sets:
                    question_results.append((question_result, fa))
                    unique_fa_sets.add(fa_set)

    return question_results


templates = [
    "Why is {food} not suitable for someone who only eats the type of foods"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_why_meat_Fruit-veg.csv", ["Question", "fa"])


Results saved to question_results_why_meat_Fruit-veg.csv


In [3]:
# Why - given shape
def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_vegetables = list(onto.Vegetable.instances())
        all_shapes = list(onto.Shape.instances())

        all_meat = ["beef", "lamb", "pork", "fish", "shrimp", "salmon"]

        question_results = []

        for shape in all_shapes:
            fruits_with_shape = [f for f in all_fruits if shape in f.has_shape]
            vegs_with_shape = [v for v in all_vegetables if shape in v.has_shape]
            all_combinations = itertools.combinations(fruits_with_shape + vegs_with_shape, 5)

            for combination in all_combinations:
                # for food in ["beef", "lamb", "pork", "fish", "shrimp", "salmon"]:
                food = random.choice(all_meat)
                template = random.choice(templates)

                question_result = template.replace('{food}', food)
                question_result = question_result.replace('{shape}', shape.name)

                fa = [f.name for f in list(combination)]
                
                question_results.append((question_result, fa))

    return question_results

templates = [
    "Why is {food} not suitable for someone who only eats the {shape} foods"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_why_meat_Shapes.csv", ["Question", "fa"])



Results saved to question_results_why_meat_Shapes.csv


In [4]:
# Why - given colour
def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_vegetables = list(onto.Vegetable.instances())
        all_colours = list(onto.Color.instances())

        all_meat = ["beef", "lamb", "pork", "fish", "shrimp", "salmon"]

        question_results = []

        for colour in all_colours:
            fruits_with_colour = [f for f in all_fruits if colour in f.has_color]
            vegs_with_colour = [v for v in all_vegetables if colour in v.has_color]
            all_combinations = itertools.combinations(fruits_with_colour + vegs_with_colour, 5)

            for combination in all_combinations:
                food = random.choice(all_meat)
                template = random.choice(templates)

                question_result = template.replace('{food}', food)
                question_result = question_result.replace('{colour}', colour.name)

                fa = [f.name for f in list(combination)]
                
                question_results.append((question_result, fa))

    return question_results

templates = [
    "Why is {food} not suitable for someone who only eats the {colour} foods"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_why_meat_Colour.csv", ["Question", "fa"])



Results saved to question_results_why_meat_Colour.csv


In [5]:
# Why - given seed
def generate_all_question_results(templates):
    with onto:
        all_fruits_veg = list(onto.Fruit.instances()) + list(onto.Vegetable.instances())

        all_meat = ["beef", "lamb", "pork", "fish", "shrimp", "salmon"]

        question_results = []

        fruits_veg_with_seed = [f for f in all_fruits_veg if f.has_seed_inside]
        all_combinations = itertools.combinations(fruits_veg_with_seed, 5)

        for combination in all_combinations:
            food = random.choice(all_meat)
            template = random.choice(templates)

            question_result = template.replace('{food}', food)

            fa = [f.name for f in list(combination)]
            
            question_results.append((question_result, fa))

    return question_results

templates = [
    "Why is {food} not suitable for someone who only eats the foods containing seeds inside"
]

results = generate_all_question_results(templates)

save_as_csv(results, "question_results_why_meat_Seed.csv", ["Question", "fa"])



Results saved to question_results_why_meat_Seed.csv


In [ ]:
import itertools

def generate_all_question_results(templates):
    with onto:
        all_fruits = list(onto.Fruit.instances())
        all_vegetables = list(onto.Vegetable.instances())
        all_shapes = list(onto.Shape.instances())

        question_results = []

        for shape in all_shapes:
            # Combine fruits and vegetables with the specified shape
            fruits_with_shape = [f for f in all_fruits if shape in f.has_shape]
            vegs_with_shape = [v for v in all_vegetables if shape in v.has_shape]

            # Generate all unique combinations of 5 items
            all_combinations = itertools.product(fruits_with_shape + vegs_with_shape, repeat=5)

            for combination in all_combinations:
                # Flatten the combination and get the first 5 items
                fa = [food.name for sublist in combination for food in sublist][:5]

                # Create the question result using the template
                template = random.choice(templates)
                question_result = template.replace('{food}', ' '.join(food.name for food in itertools.chain.from_iterable(combination)))
                question_result = question_result.replace('{shape}', shape.name)

                question_results.append((question_result, fa))

    return question_results

generate_all_question_results([])
